In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

df = pd.read_excel('./menu.xlsx',
                   header=0,
                   dtype=str).fillna('')

list_url = df['Url'].tolist()

print('总数量：' + str(len(list_url)))
print()

# = = = = = = = = = = = = = = =

df = pd.DataFrame()

# = = = = = = = = = = = = = = =

import requests

from bs4 import BeautifulSoup

from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for url in list_url:
    work.put_nowait(url)

def crawler():
    global df
    
    while not work.empty():
        url = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(6):
            try:    
                resp = requests.get(url,
                                    headers=UA.get_User_Agent_Requests(),
                                     proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(resp, 'lxml')
                
                # = = = = = = = = = = = = = = =
                
                html = etree.HTML(str(soup))

                title = html.xpath('//div[@class="product-details"]/div[@class="row"]/div[@class="col-12"]/div/h1/text()')[0].strip()
                
                # = = = = = = = = = = = = = = =
                
                sku = resp.split('window.productId = ')[1].split(';')[0]

                vehicle_url = 'https://www.1aauto.com/catalog/product/fit?skuId=' + sku + '&pathName=' + url.replace('https://www.1aauto.com', '').replace('/', '%2F')

                # = = = = = = = = = = = = = = =
                
                src = html.xpath('//img[@id="main-img"]/@src')[0].strip()
                
                # = = = = = = = = = = = = = = =
                
                list_li = html.xpath('//div[@id="product-description-tab"]/div[1]/ul/li')
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'status': 'ok',
                                         '序号': '',
                                         'Title': title,
                                         'Vehicle': '',
                                         'OE': '',
                                         'Pic': '',
                                         'Url': url,
                                         'Vehicle_Url': vehicle_url,
                                         'Src': src}])

                for li in list_li:
                    list_span = li.xpath('./span')[1:]
                    text_span = ''
                    for span in list_span:
                        text_span += span.xpath('./text()')[0].strip() + '\n'
                    df_temp.loc[0, li.xpath('./span')[0].xpath('./text()')[0].strip()[:-1]] = text_span.strip()

                status = 'ok'
                
                break
                
            except:
                time.sleep(1)
                continue

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Url': url}])

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(url +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()) + ' - 当前时间：' + datetime.now().strftime('%H:%M:%S'))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

df = df.drop_duplicates(ignore_index=True)

print()
print('爬虫输出中')
df.to_excel('./part_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
print()
print('爬虫结束')

总数量：6589

https://www.1aauto.com/1996-00-chevrolet-k3500-gmc-k3500-front-semi-metallic-brake-pad-and-rotor-kit-with-calipers-and-hoses-trq-bka29768/i/1abfs16341  <->  [ok] - 剩余数量：6569 - 当前时间：10:09:16
https://www.1aauto.com/1990-02-chevrolet-astro-gmc-safari-front-ceramic-brake-pad-and-rotor-kit-with-calipers-trq-bka18389/i/1abfs08752  <->  [ok] - 剩余数量：6568 - 当前时间：10:09:16
https://www.1aauto.com/1998-02-chevrolet-prizm-toyota-corolla-front-semi-metallic-brake-pad-and-rotor-kit-with-calipers-trq-bka18480/i/1abfs08698  <->  [ok] - 剩余数量：6567 - 当前时间：10:09:16
https://www.1aauto.com/1996-00-chevrolet-k3500-gmc-k3500-front-ceramic-brake-pad-and-rotor-kit-with-calipers-and-hoses-trq-bka29767/i/1abfs16340  <->  [ok] - 剩余数量：6566 - 当前时间：10:09:16
https://www.1aauto.com/1990-02-chevrolet-astro-gmc-safari-front-semi-metallic-brake-pad-and-rotor-kit-with-calipers-trq-bka18434/i/1abfs08776  <->  [ok] - 剩余数量：6565 - 当前时间：10:09:16
https://www.1aauto.com/1998-02-chevrolet-prizm-toyota-corolla-front-ceramic

https://www.1aauto.com/2002-04-honda-cr-v-rear-ceramic-brake-pad-and-rotor-kit-with-calipers-trq-bka18509/i/1abfs08691  <->  [ok] - 剩余数量：6525 - 当前时间：10:09:23
https://www.1aauto.com/1999-04-jeep-grand-cherokee-rear-ceramic-brake-pad-and-rotor-kit-with-calipers-trq-bka14378/i/1abfs04506  <->  [ok] - 剩余数量：6524 - 当前时间：10:09:23
https://www.1aauto.com/2009-lincoln-navigator-from-7-01-09-production-date-front-semi-metallic-brake-pad-and-rotor-kit-with-calipers-trq-bka25499/i/1abfs13469  <->  [ok] - 剩余数量：6523 - 当前时间：10:09:23
https://www.1aauto.com/2000-04-toyota-avalon-rear-ceramic-brake-pad-and-rotor-kit-with-calipers-trq-bka19145/i/1abfs09267  <->  [ok] - 剩余数量：6522 - 当前时间：10:09:23
https://www.1aauto.com/2002-hyundai-elantra-to-10-31-01-production-date-front-ceramic-performance-brake-pad-and-rotor-kit-with-calipers-trq-performance-bka26190/i/1apbs04906  <->  [ok] - 剩余数量：6521 - 当前时间：10:09:23
https://www.1aauto.com/2003-nissan-maxima-from-2-01-03-production-date-rear-semi-metallic-brake-pad-and

https://www.1aauto.com/2001-02-chevrolet-gmc-front-driver-and-passenger-side-4-piece-brake-caliper-set-with-hoses-trq-cla41344/i/1abcs00858  <->  [ok] - 剩余数量：6482 - 当前时间：10:09:30
https://www.1aauto.com/1998-02-ford-lincoln-mercury-front-and-rear-ceramic-brake-pad-and-rotor-kit-trq-bka11537/i/1abfs02211  <->  [ok] - 剩余数量：6481 - 当前时间：10:09:30
https://www.1aauto.com/1998-02-chevrolet-prizm-toyota-corolla-front-driver-side-brake-caliper-with-mounting-bracket-trq-cla34664/i/1abcr00217  <->  [ok] - 剩余数量：6480 - 当前时间：10:09:30
https://www.1aauto.com/2001-02-toyota-4runner-with-16-inch-wheels-with-caliper-casting-%23s13wm-front-ceramic-brake-pad-and-rotor-kit-with-calipers-and-hoses-trq-bka29398/i/1abfs16033  <->  [ok] - 剩余数量：6479 - 当前时间：10:09:31
https://www.1aauto.com/1990-02-chevrolet-astro-gmc-safari-front-ceramic-brake-pad-and-rotor-kit-with-calipers-trq-bka18366/i/1abfs08774  <->  [ok] - 剩余数量：6478 - 当前时间：10:09:31
https://www.1aauto.com/2000-04-toyota-avalon-rear-passenger-side-brake-caliper

https://www.1aauto.com/1996-00-toyota-4runner-with-16-inch-wheels-with-caliper-casting-%23s13wm-front-semi-metallic-performance-brake-pad-and-rotor-kit-with-calipers-and-hoses-trq-performance-bka34353/i/1apbs07435  <->  [ok] - 剩余数量：6435 - 当前时间：10:09:41
https://www.1aauto.com/1996-00-dodge-chrysler-plymouth-front-ceramic-brake-pad-and-rotor-kit-with-calipers-and-hoses-trq-bka29570/i/1abfs16142  <->  [ok] - 剩余数量：6434 - 当前时间：10:09:41
https://www.1aauto.com/1996-00-dodge-chrysler-plymouth-front-driver-and-passenger-side-semi-metallic-performance-brake-pad-and-rotor-kit-trq-performance-bka21617/i/1apbs04135  <->  [ok] - 剩余数量：6433 - 当前时间：10:09:41
https://www.1aauto.com/1996-00-dodge-chrysler-plymouth-front-semi-metallic-brake-pad-and-rotor-kit-with-calipers-and-hoses-trq-bka29571/i/1abfs16143  <->  [ok] - 剩余数量：6432 - 当前时间：10:09:42
https://www.1aauto.com/1996-00-dodge-caravan-plymouth-voyager-front-ceramic-brake-pad-and-rotor-kit-with-calipers-and-hoses-trq-bka29572/i/1abfs16144  <->  [ok] - 

https://www.1aauto.com/2003-04-honda-accord-coupe-l4-2.4l-manual-transmission-front-driver-and-passenger-side-ceramic-brake-pad-and-rotor-kit-with-calipers-and-hoses-trq-bka29888/i/1abfs16460  <->  [ok] - 剩余数量：6402 - 当前时间：10:09:50
https://www.1aauto.com/2003-04-cadillac-seville-front-ceramic-brake-pad-and-rotor-kit-with-calipers-and-hoses-trq-bka29776/i/1abfs16349  <->  [ok] - 剩余数量：6401 - 当前时间：10:09:50
https://www.1aauto.com/2003-04-honda-accord-coupe-l4-2.4l-manual-transmission-front-semi-metallic-performance-brake-pad-and-rotor-kit-with-calipers-trq-performance-bka26239/i/1apbs04955  <->  [ok] - 剩余数量：6400 - 当前时间：10:09:51
https://www.1aauto.com/2003-04-honda-accord-coupe-l4-2.4l-manual-transmission-front-driver-and-passenger-side-semi-metallic-brake-pad-and-rotor-kit-with-calipers-and-hoses-trq-bka29889/i/1abfs16461  <->  [ok] - 剩余数量：6399 - 当前时间：10:09:51
https://www.1aauto.com/2002-04-honda-cr-v-rear-ceramic-brake-pad-and-rotor-kit-with-calipers-and-hoses-trq-bka29505/i/1abfs16077  <-

...

https://www.1aauto.com/honda-pilot-acura-mdx-rear-driver-and-passenger-side-semi-metallic-performance-brake-pad-and-rotor-kit-with-calipers-trq-performance-bka35734/i/1apbs07518  <->  [ok] - 剩余数量：187 - 当前时间：10:38:32
https://www.1aauto.com/vw-beetle-golf-jetta-front-semi-metallic-performance-brake-pad-and-rotor-kit-with-calipers-trq-performance-bka35735/i/1apbs07519  <->  [ok] - 剩余数量：186 - 当前时间：10:38:32
https://www.1aauto.com/nissan-murano-quest-rear-semi-metallic-performance-brake-pad-and-rotor-kit-with-performance-calipers-trq-performance-bka35819/i/1apbs07603  <->  [ok] - 剩余数量：185 - 当前时间：10:38:32
https://www.1aauto.com/mitsubishi-front-semi-metallic-performance-brake-pad-and-rotor-kit-with-calipers-trq-performance-bka35813/i/1apbs07597  <->  [ok] - 剩余数量：184 - 当前时间：10:38:33
https://www.1aauto.com/honda-accord-acura-tsx-rear-semi-metallic-performance-brake-pad-and-rotor-kit-with-performance-calipers-trq-performance-bka35814/i/1apbs07598  <->  [ok] - 剩余数量：183 - 当前时间：10:38:33
https://www

https://www.1aauto.com/chevrolet-gmc-rear-driver-and-passenger-side-2-piece-brake-caliper-set-raybestos-element-3-rabcs00060/i/rabcs00060  <->  [ok] - 剩余数量：142 - 当前时间：10:38:41
https://www.1aauto.com/ford-lincoln-front-driver-and-passenger-side-2-piece-brake-caliper-set-raybestos-element-3-rabcs00051/i/rabcs00051  <->  [ok] - 剩余数量：141 - 当前时间：10:38:41
https://www.1aauto.com/toyota-pontiac-scion-front-driver-and-passenger-side-2-piece-brake-caliper-set-raybestos-element-3-rabcs00066/i/rabcs00066  <->  [ok] - 剩余数量：140 - 当前时间：10:38:41
https://www.1aauto.com/honda-acura-front-driver-and-passenger-side-2-piece-brake-caliper-set-raybestos-element-3-rabcs00017/i/rabcs00017  <->  [ok] - 剩余数量：139 - 当前时间：10:38:41
https://www.1aauto.com/ford-f150-f150-heritage-front-driver-and-passenger-side-2-piece-brake-caliper-set-raybestos-element-3-rabcs00055/i/rabcs00055  <->  [ok] - 剩余数量：138 - 当前时间：10:38:42
https://www.1aauto.com/honda-accord-acura-tsx-rear-driver-and-passenger-side-2-piece-brake-caliper-set

https://www.1aauto.com/ford-rear-passenger-side-brake-caliper-with-mounting-bracket-raybestos-element-3-frc11005n/i/rabcr00105  <->  [ok] - 剩余数量：94 - 当前时间：10:38:51
https://www.1aauto.com/jeep-dodge-ram-vw-chrysler-front-ceramic-brake-pads-raybestos-mgd1273ch/i/rabps00102  <->  [ok] - 剩余数量：93 - 当前时间：10:38:51
https://www.1aauto.com/toyota-4runner-fj-cruiser-tacoma-front-passenger-side-brake-caliper-raybestos-element-3-frc11553n/i/rabcr00117  <->  [ok] - 剩余数量：92 - 当前时间：10:38:51
https://www.1aauto.com/ford-lincoln-driver-and-passenger-side-2-piece-wheel-bearing-and-hub-assembly-set-motorcraft-mcshs00016/i/mcshs00016  <->  [ok] - 剩余数量：91 - 当前时间：10:38:52
https://www.1aauto.com/chevrolet-buick-pontiac-olds-rear-passenger-side-brake-caliper-with-mounting-bracket-raybestos-element-3-frc10842n/i/rabcr00100  <->  [ok] - 剩余数量：90 - 当前时间：10:38:52
https://www.1aauto.com/ford-front-driver-and-passenger-side-2-piece-brake-caliper-set-raybestos-rabcs00072/i/rabcs00072  <->  [ok] - 剩余数量：89 - 当前时间：10:38:5

https://www.1aauto.com/chevrolet-buick-pontiac-rear-driver-side-brake-caliper-with-mounting-bracket-raybestos-element-3-frc11841n/i/rabcr00083  <->  [ok] - 剩余数量：46 - 当前时间：10:39:02
https://www.1aauto.com/ford-mazda-mercury-front-driver-side-brake-caliper-with-mounting-bracket-raybestos-element-3-frc11524n/i/rabcr00116  <->  [ok] - 剩余数量：45 - 当前时间：10:39:02
https://www.1aauto.com/ford-front-driver-side-brake-caliper-with-mounting-bracket-raybestos-element-3-frc11510n/i/rabcr00114  <->  [ok] - 剩余数量：44 - 当前时间：10:39:03
https://www.1aauto.com/ford-mazda-mercury-front-passenger-side-brake-caliper-with-mounting-bracket-raybestos-element-3-frc11523n/i/rabcr00115  <->  [ok] - 剩余数量：43 - 当前时间：10:39:03
https://www.1aauto.com/ford-e150-e250-e350-super-duty-rear-passenger-side-brake-caliper-with-mounting-bracket-raybestos-element-3-frc12043n/i/rabcr00135  <->  [ok] - 剩余数量：42 - 当前时间：10:39:03
https://www.1aauto.com/dodge-ram-chrysler-front-brake-caliper-with-mounting-bracket-raybestos-element-3-frc11896n

https://www.1aauto.com/rear-driver-side-brake-caliper-with-mounting-bracket-raybestos-element-3-frc11331n/i/rabcr00011  <->  [ok] - 剩余数量：0 - 当前时间：10:39:13
https://www.1aauto.com/chevrolet-pontiac-olds-front-passenger-side-brake-caliper-with-mounting-bracket-raybestos-element-3-frc10906n/i/rabcr00002  <->  [ok] - 剩余数量：0 - 当前时间：10:39:13
https://www.1aauto.com/chevrolet-buick-pontiac-olds-rear-driver-side-brake-caliper-with-mounting-bracket-raybestos-element-3-frc10841n/i/rabcr00099  <->  [ok] - 剩余数量：0 - 当前时间：10:39:13
https://www.1aauto.com/chevrolet-dodge-gmc-cadillac-front-passenger-side-brake-caliper-raybestos-element-3-frc4417n/i/rabcr00141  <->  [ok] - 剩余数量：0 - 当前时间：10:39:14
https://www.1aauto.com/chevrolet-pontiac-olds-front-driver-side-brake-caliper-with-mounting-bracket-raybestos-element-3-frc10905n/i/rabcr00001  <->  [ok] - 剩余数量：0 - 当前时间：10:39:14
https://www.1aauto.com/chevrolet-gmc-cadillac-rear-driver-side-brake-caliper-with-mounting-bracket-raybestos-element-3-frc11573n/i/rabc